In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [2]:
d2 = dice_ml.Data(features={
                    'Pclass': ['First','Second','Third'],
                    'Sex': ['Male','Female'],
                    'Age': [0,90],
                    'Fare': [0,512],
                    'Embarked': ['Southampton', 'Cherbourg', 'Queenstown'],
                    'Relatives': [0,10]
                   },
         outcome_name='class')

In [3]:
train_df = pd.read_csv('titanic_train.csv')
test_df = pd.read_csv('titanic_test.csv')
data = train_df.append(test_df)

#print(data)
#data.head()

d1 = dice_ml.Data(dataframe=data,
                 continuous_features=['Age', 'Fare', 'Relatives'],
                 outcome_name='class')

In [5]:
sess = tf.InteractiveSession()

ann_model = tf.keras.models.load_model("titanic_ann.h5")


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [6]:
#print(ann_model.evaluate(X_test, y_test, verbose=0)[1])
m = dice_ml.Model(model=ann_model)
#ML_modelpath = dice_ml.utils.helpers.get_adult_income_modelpath()
#m = dice_ml.Model(model_path= ML_modelpath)

In [13]:
# initiate DiCE
exp = dice_ml.Dice(d2, m)
#exp = dice_ml.dice_interfaces.dice_tensorflow.DiceTensorFlow(d2, m)

<tf.Variable 'Variable_6:0' shape=(1, 11) dtype=float32_ref>


In [14]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'Pclass':'First', 
                  'Sex':'Male', 
                  'Age':20, 
                  'Fare':5, 
                  'Embarked':'Southampton',
                  'Relatives': 1}
#transform the query_instance into a format that ann_model.predict_proba(p) can use and which has the same prediction prob as dice-visualization

In [15]:
#print(train)
#p = np.array([[1.0, 0.3375, 0.412109, 0.285714, 0, 1, 1, 0, 0]])
#print(ann_model.predict_proba(p))
#print(exp.predict_fn(p))
test = exp.data_interface.prepare_query_instance(query_instance=query_instance, encode=True)
test = np.array([test.iloc[0].values])
print(test)
print(ann_model.predict_proba(test))

#print(train)
#print(data.loc[[377]])
#print(d.one_hot_encoded_data)

[[0.22222222 0.00976562 0.1        1.         0.         0.
  0.         1.         0.         0.         1.        ]]
[[0.637296]]


In [16]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=1, desired_class="opposite", features_to_vary=['Pclass'], proximity_weight=1.5, diversity_weight=2)

Tensor("add_10:0", shape=(1, 11), dtype=float32)
Tensor("add_10:0", shape=(1, 11), dtype=float32)
Diverse Counterfactuals found! total time taken: 00 min 01 sec


In [17]:
print(dice_exp)

In [18]:
# visualize the results
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 1)

Diverse Counterfactual set (new outcome : 0)


,Pclass,Sex,Age,Fare,Embarked,Relatives,class
0,First,Male,20.0,5.0,Southampton,1.0,0.637296


,Pclass,Sex,Age,Fare,Embarked,Relatives,class
0,Second,Male,20.0,5.0,Southampton,1.0,0.147


In [23]:
dice_exp.visualize_as_list()

Query instance (original outcome : 1)
['First', 'Male', 20.0, 5.0, 'Southampton', 0.0, 0.6650130152702332]

Diverse Counterfactual set (new outcome : 0)
['Second', 'Male', 20.0, 5.0, 'Southampton', 0.0, 0.155]
